In [ ]:
# Install required libraries
!pip install requests beautifulsoup4 google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import requests
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow


In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
from io import BytesIO
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload

# Update this path to where your service account JSON key is located
SERVICE_ACCOUNT_FILE = '/content/sunny-strategy-446220-m9-41cbb114efa5.json'

# Your Google Drive file ID for the text file that contains the start URL
# >>> UPDATE THIS WITH THE CORRECT FILE ID <<<
FILE_ID = '1K47QnRifDpGyyafWEf1yK3bBrO8VBKFj'

SCOPES = ['https://www.googleapis.com/auth/drive']

visited_urls = set()

def is_valid_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

def get_domain(url):
    return urlparse(url).netloc

def scan_website_recursively(start_url, base_domain, max_depth, current_depth=0):
    internal_links = []
    if current_depth > max_depth:
        return internal_links

    normalized_url = urljoin(start_url, urlparse(start_url).path)
    if normalized_url in visited_urls:
        return internal_links

    visited_urls.add(normalized_url)
    print(f"Crawling: {normalized_url} (Depth: {current_depth})")

    try:
        response = requests.get(normalized_url, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {normalized_url}: {e}")
        return internal_links

    soup = BeautifulSoup(response.text, 'html.parser')

    for tag in soup.find_all('a', href=True):
        href = tag['href']
        full_url = urljoin(normalized_url, href)
        clean_url = urljoin(full_url, urlparse(full_url).path)

        if not is_valid_url(clean_url):
            continue

        if get_domain(clean_url) == base_domain and clean_url not in visited_urls:
            internal_links.append(clean_url)
            internal_links += scan_website_recursively(clean_url, base_domain, max_depth, current_depth + 1)

    time.sleep(0.5)  # polite delay to avoid hammering the server
    return internal_links

def get_url_from_drive():
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    service = build('drive', 'v3', credentials=creds)

    request = service.files().get_media(fileId=FILE_ID)
    fh = BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()

    url_text = fh.getvalue().decode("utf-8").strip()
    print(f"Start URL read from Drive file: {url_text}")
    return url_text

def upload_to_drive(content):
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    service = build('drive', 'v3', credentials=creds)

    file_metadata = {'name': 'crawled_links.txt', 'mimeType': 'text/plain'}
    fh = BytesIO(content.encode('utf-8'))
    media = MediaIoBaseUpload(fh, mimetype='text/plain')
    uploaded_file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f"Uploaded crawled links file to Drive with File ID: {uploaded_file['id']}")
    return uploaded_file['id']

def main():
    start_url = get_url_from_drive()
    if not is_valid_url(start_url):
        print("Invalid URL in Drive file.")
        return

    base_domain = get_domain(start_url)
    print(f"Crawling domain: {base_domain}")

    results = scan_website_recursively(start_url, base_domain, max_depth=2)
    print(f"Found {len(results)} internal links.")

    content = "\n".join(results)
    upload_to_drive(content)

if __name__ == "__main__":
    main()


HttpError: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/1K47QnRifDpGyyafWEf1yK3bBrO8VBKFj?alt=media returned "File not found: 1K47QnRifDpGyyafWEf1yK3bBrO8VBKFj.". Details: "[{'message': 'File not found: 1K47QnRifDpGyyafWEf1yK3bBrO8VBKFj.', 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">

In [5]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
from io import BytesIO
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload

# Update this path to where your service account JSON key is located
SERVICE_ACCOUNT_FILE = '/content/sunny-strategy-446220-m9-41cbb114efa5.json'

# Your Google Drive file ID for the text file that contains the start URL

FILE_ID = '1q30D3P03lgPciN2D41xjn1jQDbk'

SCOPES = ['https://www.googleapis.com/auth/drive']

visited_urls = set()

def is_valid_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

def get_domain(url):
    return urlparse(url).netloc

def scan_website_recursively(start_url, base_domain, max_depth, current_depth=0):
    internal_links = []
    if current_depth > max_depth:
        return internal_links

    normalized_url = urljoin(start_url, urlparse(start_url).path)
    if normalized_url in visited_urls:
        return internal_links

    visited_urls.add(normalized_url)
    print(f"Crawling: {normalized_url} (Depth: {current_depth})")

    try:
        response = requests.get(normalized_url, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {normalized_url}: {e}")
        return internal_links

    soup = BeautifulSoup(response.text, 'html.parser')

    for tag in soup.find_all('a', href=True):
        href = tag['href']
        full_url = urljoin(normalized_url, href)
        clean_url = urljoin(full_url, urlparse(full_url).path)

        if not is_valid_url(clean_url):
            continue

        if get_domain(clean_url) == base_domain and clean_url not in visited_urls:
            internal_links.append(clean_url)
            internal_links += scan_website_recursively(clean_url, base_domain, max_depth, current_depth + 1)

    time.sleep(0.5)  # polite delay to avoid hammering the server
    return internal_links

def get_url_from_drive():
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    service = build('drive', 'v3', credentials=creds)

    request = service.files().get_media(fileId=FILE_ID)
    fh = BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()

    url_text = fh.getvalue().decode("utf-8").strip()
    print(f"Start URL read from Drive file: {url_text}")
    return url_text

def upload_to_drive(content):
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    service = build('drive', 'v3', credentials=creds)

    file_metadata = {'name': 'crawled_links.txt', 'mimeType': 'text/plain'}
    fh = BytesIO(content.encode('utf-8'))
    media = MediaIoBaseUpload(fh, mimetype='text/plain')
    uploaded_file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f"Uploaded crawled links file to Drive with File ID: {uploaded_file['id']}")
    return uploaded_file['id']

def main():
    start_url = get_url_from_drive()
    if not is_valid_url(start_url):
        print("Invalid URL in Drive file.")
        return

    base_domain = get_domain(start_url)
    print(f"Crawling domain: {base_domain}")

    results = scan_website_recursively(start_url, base_domain, max_depth=2)
    print(f"Found {len(results)} internal links.")

    content = "\n".join(results)
    upload_to_drive(content)

if __name__ == "__main__":
    main()

HttpError: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/1q30D3P03lgPciN2D41xjn1jQDbk?alt=media returned "File not found: 1q30D3P03lgPciN2D41xjn1jQDbk.". Details: "[{'message': 'File not found: 1q30D3P03lgPciN2D41xjn1jQDbk.', 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">